## Data Cleaning

In [21]:
import pandas as pd

In [72]:
# Load the 7259deaf70e0b097.xlsx file and drop unnecessary columns
AAPL_option = pd.read_excel('/Users/baronma/Desktop/7259deaf70e0b097.xlsx')

In [73]:
AAPL_option.head()

,Security ID,The Date of this Price,Option Symbol,Expiration Date of the Option,"C=Call, P=Put",Strike Price of the Option Times 1000,Highest Closing Bid Across All Exchanges,Lowest Closing Ask Across All Exchanges,Open Interest for the Option,Implied Volatility of the Option,Unique ID for the Option Contract,Ticker Symbol,Index Flag,Description of the Issuing Company,"(A)merican, (E)uropean, or ?"
0,101594,20120904,AAPL 120907C590000,20120907,C,590000,84.70,85.50,113,0.636391,83202292,AAPL,0,APPLE INC,A
1,101594,20120904,AAPL 120907C595000,20120907,C,595000,78.90,80.80,0,NaN,83202293,AAPL,0,APPLE INC,A
2,101594,20120904,AAPL 120907C600000,20120907,C,600000,74.70,75.15,40,NaN,83202294,AAPL,0,APPLE INC,A
3,101594,20120904,AAPL 120907C605000,20120907,C,605000,68.85,70.70,0,NaN,83202295,AAPL,0,APPLE INC,A
4,101594,20120904,AAPL 120907C610000,20120907,C,610000,64.75,65.45,8,0.495483,83202296,AAPL,0,APPLE INC,A


In [75]:
AAPL_option.drop(['Security ID', 'Option Symbol', 'Unique ID for the Option Contract', 'Ticker Symbol', 'Index Flag',
                  'Description of the Issuing Company', '(A)merican, (E)uropean, or ?','Highest Closing Bid Across All Exchanges'], axis = 1, inplace = True)

In [25]:
# Load the Stock Prices.xlsx file and filter AAPL stock ptice.
AAPL_stock = pd.read_excel('/Users/baronma/Desktop/Stock Prices.xlsx')
AAPL_stock = AAPL_stock[AAPL_stock['Ticker Symbol'] == 'AAPL']

In [26]:
# Rename the 'The Date for this Price Record' column to be the same as the AAPL option dataframe, in order to merge these two.
AAPL_stock.rename(columns = {'The Date for this Price Record': 'The Date of this Price'}, inplace = True)
AAPL_stock.head()

,Security ID,The Date of this Price,Ticker Symbol,Close (or Bid-Ask Average if Negative),Cumulative Adjustment Factor
1340,101594,20120904,AAPL,674.97,4
1341,101594,20120905,AAPL,670.23,4
1342,101594,20120906,AAPL,676.27,4
1343,101594,20120907,AAPL,680.44,4
1344,101594,20120910,AAPL,662.74,4


In [77]:
# Get the original strike price
AAPL_option['Strike Price of the Option Times 1000'] = AAPL_option['Strike Price of the Option Times 1000'].apply(lambda x: x/1000)
AAPL_option.rename(columns={'Strike Price of the Option Times 1000': 'Strike Price'}, inplace=True)

In [78]:
# Left join AAPL_stock to AAPL_option in order to add the underlying stock price to the option
AAPL = pd.merge(AAPL_option, AAPL_stock, on = ['The Date of this Price'], how = 'left')

In [80]:
AAPL.drop(['Security ID', 'Ticker Symbol', 'Cumulative Adjustment Factor'], axis = 1, inplace = True)
AAPL.rename(columns = {'Close (or Bid-Ask Average if Negative)' : 'Underlying Price'}, inplace = True)

In [81]:
AAPL.head()

,The Date of this Price,Expiration Date of the Option,"C=Call, P=Put",Strike Price,Lowest Closing Ask Across All Exchanges,Open Interest for the Option,Implied Volatility of the Option,Underlying Price
0,20120904,20120907,C,590.0,85.50,113,0.636391,674.97
1,20120904,20120907,C,595.0,80.80,0,NaN,674.97
2,20120904,20120907,C,600.0,75.15,40,NaN,674.97
3,20120904,20120907,C,605.0,70.70,0,NaN,674.97
4,20120904,20120907,C,610.0,65.45,8,0.495483,674.97


In [82]:
# Change the text date to the standard date format
AAPL['The Date of this Price'] = pd.to_datetime(AAPL['The Date of this Price'], format = '%Y%m%d')
AAPL['Expiration Date of the Option'] = pd.to_datetime(AAPL['Expiration Date of the Option'], format = '%Y%m%d')

In [83]:
# Calculate the maturity days
AAPL['maturity'] = (AAPL['Expiration Date of the Option'] - AAPL['The Date of this Price']).dt.days

In [84]:
AAPL_withoutNaN = AAPL[AAPL['Implied Volatility of the Option'].notnull()]
len(AAPL_withoutNaN)

727050

In [85]:
# One hot encoding to the 'C=Call, P=Put' column
AAPL_final = pd.get_dummies(data=AAPL, columns=['C=Call, P=Put'])
AAPL_final_2 = pd.get_dummies(data=AAPL_withoutNaN, columns=['C=Call, P=Put'])

In [86]:
AAPL_final.head()

,The Date of this Price,Expiration Date of the Option,Strike Price,Lowest Closing Ask Across All Exchanges,Open Interest for the Option,Implied Volatility of the Option,Underlying Price,maturity,"C=Call, P=Put_C","C=Call, P=Put_P"
0,2012-09-04,2012-09-07,590.0,85.50,113,0.636391,674.97,3,1,0
1,2012-09-04,2012-09-07,595.0,80.80,0,NaN,674.97,3,1,0
2,2012-09-04,2012-09-07,600.0,75.15,40,NaN,674.97,3,1,0
3,2012-09-04,2012-09-07,605.0,70.70,0,NaN,674.97,3,1,0
4,2012-09-04,2012-09-07,610.0,65.45,8,0.495483,674.97,3,1,0


In [87]:
AAPL_final.to_csv('Cleaned.csv')

In [88]:
AAPL_final_2.to_csv('Cleaned_withoutNaN.csv')